# Settings

In [ ]:
save_figs = False

# Bachelor - Covid Dataset - Exploratory Data Analysis

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import neighbors, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt




#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
pip install pytorch_lightning

In [ ]:
covid = pd.read_csv('drive/MyDrive/host_train.csv')

In [ ]:
covid.head()

Removing the `case_id` and `patient_id` columns

In [ ]:
covid = covid[ [name for name in covid.columns if name not in ['case_id', 'patientid', 'Hospital', 'Hospital_city', 'City_Code_Patient'] ] ]
covid = covid.dropna()

In [ ]:
covid.head()

Extracting the response variable class names.

In [ ]:
s = [ str(x) for x in covid['Stay_Days'].unique()]
s.sort()

Plotting the distribution of the classes.

In [ ]:
plt.style.available


In [ ]:
plt.style.core.available

In [ ]:
plt.style.use('ggplot')

plt.bar( s , covid['Stay_Days'].value_counts().sort_index())
plt.xticks(rotation = 90)
#plt.plot('Original reponse class distribution')
plt.show()

Reducing the number of classes in the response.

Patients, who were in the hospital for more than 40 days are now in one group.

This makes the groups more balanced, as seen in the bar plot below.

In [ ]:
covid_reduced = covid.copy()
covid_reduced['Stay_Days'] = covid['Stay_Days'].replace(to_replace=['51-60','61-70', '71-80', '81-90', '91-100','More than 100 Days'],
                                                        value='51+' , inplace=False )

covid_reduced['Stay_Days'] = covid_reduced['Stay_Days'].replace(to_replace=['0-10', '11-20'],
                                                        value='0-20' , inplace=False )


covid_reduced['Stay_Days'] = covid_reduced['Stay_Days'].replace(to_replace=['31-40', '41-50'],
                                                        value='31-50' , inplace=False )

y_two_class = covid_reduced['Stay_Days'].replace(to_replace=['0-20', '21-30', '31-50'],
                                                        value='0-50' , inplace=False )

In [ ]:
plt.bar(['0-50', '51+'], y_two_class.value_counts().sort_index())

In [ ]:
sd_classes_reduced = [ str(x) for x in covid_reduced['Stay_Days'].unique()]
sd_classes_reduced.sort()

sd_classes_reduced_two_class = ['0-50', '51+']

In [ ]:
plt.bar( sd_classes_reduced , covid_reduced['Stay_Days'].value_counts().sort_index())
plt.xticks(rotation = 90)
plt.title('Adjusted response class distribution')
plt.show()

In [ ]:
covid = covid_reduced.copy()

Changing the type of ordinal categorical variables/columns into the Ordered Categorical DataType.

In [ ]:
from pandas.api.types import CategoricalDtype

ordered_cat_type = CategoricalDtype(categories=sd_classes_reduced, ordered=True)

In [ ]:
covid['Stay_Days'] = covid['Stay_Days'].astype(ordered_cat_type)
covid['Bed_Grade'] = covid['Bed_Grade'].astype('int64') # or as ordered type

In [ ]:
covid.info()
# covid.head()

In [ ]:
nominal_categoricals = ['Hospital_type', 'Hospital_region',
      'Department', 'Ward_Type',
       'Ward_Facility', 'Type of Admission']

In [ ]:
illness_sev_levels = list(covid['Illness_Severity'].unique())
illness_sev_levels.reverse()
illness_sev_levels

age_levels = list(covid['Age'].unique())
age_levels.sort()
age_levels

In [ ]:
ordinal_categoricals = [
       'Illness_Severity', 'Age',
       'Stay_Days']

ordered_cat_type_illness_sev = CategoricalDtype(categories=illness_sev_levels, ordered=True)
ordered_cat_type_age = CategoricalDtype(categories=age_levels, ordered=True)
# maybe add bed grade as ordinal cat



In [ ]:
for col in nominal_categoricals:
  covid[col] = covid[col].astype("category")

covid['Illness_Severity'] = covid['Illness_Severity'].astype(ordered_cat_type_illness_sev)
covid['Age'] = covid['Age'].astype(ordered_cat_type_age)


Should be CategoricalDtype

In [ ]:
covid['Age'].dtype

#covid['Stay_Days'].dtype

In [ ]:
# from mord import OrdinalRidge

In [ ]:
#o_r = OrdinalRidge(alpha=0.1)

Initializing X (features/predictors/covariates) and y (labels/response/dependent_variable)

In [ ]:
X = covid.loc[:, covid.columns != 'Stay_Days']
y = covid['Stay_Days']

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

`.get_dummies()` encodes ordinal predictors as nominal, therefore we get 127 columns instead of 116.

In [ ]:
pd.get_dummies(X).shape

Defining the list with names of all categorical variables

In [ ]:
categoricals = nominal_categoricals.copy()
categoricals.extend(ordinal_categoricals)

Defining a list of ordinal **predictors**

In [ ]:
ordinal_predictors = [x for x in ordinal_categoricals if x != 'Stay_Days']

Encoding ordinal predictors with `OrdinalEncoder` and nominal predictors with `OneHotEncoder`

In [ ]:
oe_sd = OrdinalEncoder()
oe_df = pd.DataFrame( oe_sd.fit_transform(X[  ordinal_predictors ]) )


ohe = OneHotEncoder()
ohe_df =  pd.DataFrame( ohe.fit_transform(X[ nominal_categoricals  ]).toarray() )


Assigning column names to the newly created dataframe.

In [ ]:
oe_df.columns = oe_sd.feature_names_in_

Dropping all ordinal categorical columns from X.

In [ ]:
X = X.drop(labels = list(oe_sd.feature_names_in_) , axis=1)

In [ ]:
X.head()

Adding the ordinal encoded categorical columns.

In [ ]:
# replacing the index so that concatenation works
oe_df.index = X.index
X_oe = pd.concat([ X, oe_df ], axis=1)

Removing all nominal cat. columns from X_oe.

In [ ]:
X_oe_oh = X_oe.drop(labels = list(ohe.feature_names_in_) , axis=1)

Adding the one hot encoded nominal cat. columns to the dataframe.

In [ ]:
ohe_df.index = X_oe_oh.index

ohe_df.columns = ohe.get_feature_names_out()
X_oe_oh = pd.concat([ X_oe_oh, ohe_df ], axis=1)

Nr of columns must be 36.

In [ ]:
assert( X_oe_oh.shape[1] == 36 )

# Data Preprocessing

## Ordinal and Label encoding the data

In [ ]:
# Label encoder for the (ordinal) response
le_response = LabelEncoder()
le_response_two_class = LabelEncoder()
response = pd.Series(le_response.fit_transform(y))
response_two_class = pd.Series(le_response_two_class.fit_transform(y_two_class))

## Transforming the response into wanted NN outputs

In [ ]:
r_df = pd.DataFrame()
for i in range( len(s) ):
  temp = (response > i).astype('int64')
  r_df = pd.concat( [r_df , temp] , axis=1)


In [ ]:
pd.concat( [ pd.DataFrame() , (response > 10).astype('int64') ] , axis=1 ).values.sum()

In [ ]:
# [11100] -> 4 , [11111] -> 6 [11000] -> 3  -

r_df

## Scaling final X and PCA

In [ ]:
sc = StandardScaler()

X_sc = sc.fit_transform(X_oe_oh)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA()

X_pca = pca.fit_transform(X_sc)

In [ ]:
plt.plot(pca.explained_variance_ratio_[0:100])
plt.grid()
plt.title('PCA - Scree plot')

In [ ]:
plt.plot(pca.explained_variance_ratio_.cumsum())
plt.grid()
plt.title('Cumulative explained variance')

In order to explain 80% of the variance we can use around 20 first PCs

In [ ]:
X_reduced = X_pca[:, :20]

In [ ]:
pip install coral-pytorch

## Undersampling so that the two class response is balanced

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss


us = NearMiss()
xres , yres = us.fit_resample(pd.DataFrame(X_sc), response_two_class)

# Keras model practice

This section can be ignored.


In [ ]:
#from keras.models import Sequential
#from keras.layers import Dense

In [ ]:
#model = Sequential()
#model.add(Dense(12, input_dim=60, activation='relu'))
#model.add(Dense(8, activation='relu'))
#model.add(Dense(11, activation='sigmoid'))

In [ ]:
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# X_reduced[:10000, :].shape[0] - r_df.loc[:10000,:].shape[0]

In [ ]:
#model.fit(X_reduced[:100001, :], r_df.loc[:100000,:], epochs=2, batch_size=10)

In [ ]:
#_, accuracy = model.evaluate(X_reduced[10001:200002 ,:], r_df.loc[10000:200000, :])

In [ ]:
#res = model.predict(X_reduced[10000:20002,:])

In [ ]:
#from sklearn.metrics import confusion_matrix

#pred_df = pd.DataFrame(res.round())
#true_df = r_df.loc[10000:20001, :]



In [ ]:
#true_df.columns = range(11)

In [ ]:
#true_df.columns

In [ ]:
#true_df.head()
#pred_df.head()
#true_df.loc[:, 0]

#cm = confusion_matrix(true_df.loc[:, 0], pred_df.loc[:, 0])

In [ ]:
#from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
#cm_displ = ConfusionMatrixDisplay(cm).plot()

In [ ]:
# pip install ann_visualizer

In [ ]:
#from ann_visualizer.visualize import ann_viz
#ann_viz(model, title="My first neural network",view=True, filename="visualized")


# CORAL : Rank consistent ord. reg.

Defining the hyperparameters.

`HIDDEN_UNITS` denotes a list containing the number of neurons of each hidden layer.

It suffices to change the code here and execute `Runtime > Run after` to get the new results (along with the old ones) in the TensorBoard chart.

In [ ]:
BATCH_SIZE = 1024
NUM_EPOCHS = 15
LEARNING_RATE = 0.025 # 0.1 too big  , 0.001 too small (found with optuna plugin for hyperparameter optimization)
NUM_WORKERS = 0
HIDDEN_UNITS = [36, 36, 36, 36, 18]
HIDDEN_UNITS_CLASSIC = HIDDEN_UNITS


## Implementing the ordinal NN model (as described in CORAL manual)

Implementing custom pytorch module (multilayer perceptron)

Hidden units are given over an iterable and activation function

In [ ]:
import torch
from coral_pytorch.layers import CoralLayer


# Regular PyTorch Module
class MultiLayerPerceptron(torch.nn.Module):
    def __init__(self, input_size, hidden_units, num_classes):
        super().__init__()

        # num_classes is used by the CORAL loss function
        self.num_classes = num_classes

        # Initialize MLP layers
        all_layers = []
        for hidden_unit in hidden_units:
            layer = torch.nn.Linear(input_size, hidden_unit)
            all_layers.append(layer)
            all_layers.append(torch.nn.Sigmoid())
            input_size = hidden_unit

        # CORAL: output layer -------------------------------------------
        # Regular classifier would use the following output layer:
        # output_layer = torch.nn.Linear(hidden_units[-1], num_classes)

        # We replace it by the CORAL layer:
        output_layer = CoralLayer(size_in=hidden_units[-1],
                                  num_classes=num_classes)
        # ----------------------------------------------------------------

        all_layers.append(output_layer)
        self.model = torch.nn.Sequential(*all_layers)

    def forward(self, x):
        x = self.model(x)
        return x

Defining the custom `LightningMLP` (as a *Wrapper* class for PyTorch Models) class with:


* Metrics for training, testing and validation
* Optimizer
* Training Step
* Test Step
* Validation Step
* Shared Step (which is the first part of all three) - *returns loss, predicted labels and true labels*
* The three steps differ only in the metrics they calculate (which are defined in the `__init__` method) and logging these metrics.



### Custom Metric for sensitivity, specificity ...

In [ ]:
from torchmetrics import Metric

class MySensitivity(Metric):
    def __init__(self, num_classes, dist_sync_on_step=False):
        super().__init__(dist_sync_on_step=dist_sync_on_step)

        self.num_classes = num_classes
        self.tp = np.zeros( self.num_classes , dtype=int)
        self.tn = np.zeros( self.num_classes , dtype=int)
        self.fp = np.zeros( self.num_classes , dtype=int)
        self.fn = np.zeros( self.num_classes , dtype=int)

        # self.add_state("tp", default=torch.tensor([0,0,0,0]))
        # self.add_state("tn", default=torch.tensor([0,0,0,0]))

    def update(self, preds: torch.Tensor, target: torch.Tensor):
        assert preds.shape == target.shape

        preds = preds.cpu()
        target = target.cpu()
        for m  in range(self.num_classes): # for each class 1 to 4
          TP = tf.reduce_sum(tf.cast( ((preds == m) & (target == m) ) , tf.float32)).cpu().numpy()
          TN = tf.reduce_sum(tf.cast( ((preds != m) & (target != m) ) , tf.float32)).cpu().numpy()
          FP = tf.reduce_sum(tf.cast( ((preds == m) & (target != m) ) , tf.float32)).cpu().numpy()
          FN = tf.reduce_sum(tf.cast( ((preds != m) & (target == m) ) , tf.float32)).cpu().numpy()

          self.tp[m] += TP
          self.tn[m] += TN
          self.fp[m] += FP
          self.fn[m] += FN


    def compute(self):
        return self.tp / (self.tp + self.fn)

    def compute_sensitivity(self):
        return self.compute()

    def compute_specificity(self):
        return self.tn/ (self.fp + self.tn)

    # PPV == Precision
    def compute_precision(self):
        return self.tp / (self.tp + self.fp)

    def compute_npv(self):
        return self.tn / (self.tn + self.fn)

### Custom Metric for AUC

In [ ]:
#np.append(np.empty((0,0)) , np.array([[1,2,3]]) , axis=0)

In [ ]:
# from torchmetrics import Metric
# from sklearn.metrics import roc_curve, roc_auc_score, auc

# class MyAUC(Metric):
#     def __init__(self, dist_sync_on_step=False):
#         super().__init__(dist_sync_on_step=dist_sync_on_step)

#         self.predictions = np.empty(0,dtype=int)
#         self.targets = np.empty(0, dtype=int)


#     def update(self, preds: torch.Tensor, target: torch.Tensor):
#         assert preds.shape == target.shape

#         self.predictions = np.append(self.predictions, preds.numpy() )
#         self.targets = np.append(self.targets, target.numpy() )



#     def compute(self):
#         return roc_auc_score(
#             self.targets, self.predictions, multi_class="ovo", average="macro")


### Lightning Module (MLP)

In [ ]:
from coral_pytorch.losses import coral_loss
from coral_pytorch.dataset import levels_from_labelbatch
from coral_pytorch.dataset import proba_to_label

import pytorch_lightning as pl
import torchmetrics


# LightningModule that receives a PyTorch model as input
class LightningMLP(pl.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()

        self.learning_rate = learning_rate
        # The inherited PyTorch module
        self.model = model

        # Save settings and hyperparameters to the log directory
        # but skip the model parameters
        self.save_hyperparameters(ignore=['model'])

        # Set up attributes for computing the MAE
        self.train_mae = torchmetrics.MeanAbsoluteError()
        self.train_acc = torchmetrics.Accuracy()
        self.valid_mae = torchmetrics.MeanAbsoluteError()
        self.valid_acc = torchmetrics.Accuracy()
        self.valid_auc = torchmetrics.AUC(reorder=True)
        self.test_mae = torchmetrics.MeanAbsoluteError()
        self.valid_kappa_unweighted = torchmetrics.CohenKappa(num_classes=self.model.num_classes)
        self.valid_kappa_lin_weighted = torchmetrics.CohenKappa(num_classes=self.model.num_classes, weights='linear')
        self.valid_kappa_sq_weighted = torchmetrics.CohenKappa(num_classes=self.model.num_classes, weights='quadratic')
        #self.test_specificity = torchmetrics.Specificity(num_classes=self.model.num_classes, average='none')
        self.valid_sensitivity = MySensitivity(num_classes = self.model.num_classes)

    # Defining the forward method is only necessary
    # if you want to use a Trainer's .predict() method (optional)
    def forward(self, x):
        return self.model(x)

    # A common forward step to compute the loss and labels
    # this is used for training, validation, and testing below
    def _shared_step(self, batch):
        features, true_labels = batch

        # Convert class labels for CORAL ------------------------
        levels = levels_from_labelbatch(
            true_labels, num_classes=self.model.num_classes)
        # -------------------------------------------------------

        logits = self(features)

        # CORAL Loss --------------------------------------------
        # A regular classifier uses:
        # loss = torch.nn.functional.cross_entropy(logits, true_labels)
        loss = coral_loss(logits, levels.type_as(logits))
        # -------------------------------------------------------

        # CORAL Prediction to label -----------------------------
        # A regular classifier uses:
        # predicted_labels = torch.argmax(logits, dim=1)
        probas = torch.sigmoid(logits)
        predicted_labels = proba_to_label(probas)
        # -------------------------------------------------------
        return loss, true_labels, predicted_labels

    def training_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        self.log("train_loss", loss)
        self.train_mae(predicted_labels, true_labels)
        self.log("train_mae", self.train_mae, on_epoch=True, on_step=False)
        self.train_acc(predicted_labels, true_labels)
        self.log("train_acc", self.train_acc, on_epoch=True, on_step=False)
        return loss  # this is passed to the optimzer for training

    def validation_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        # self.log("valid_loss", loss)
        self.valid_mae(predicted_labels, true_labels)
        #self.log("valid_mae", self.valid_mae,
        #         on_epoch=True, on_step=False, prog_bar=True)
        self.valid_acc(predicted_labels, true_labels)
        self.log("valid_acc", self.valid_acc,
                 on_epoch=True, on_step=False, prog_bar=True)
        self.valid_sensitivity.update(predicted_labels, true_labels)
        self.log("valid_sens_last_class", self.valid_sensitivity.compute()[(self.model.num_classes-1)],
                  on_epoch=True, on_step=False, prog_bar=True)
        # Kappas
        self.valid_kappa_unweighted.update(predicted_labels, true_labels)
        self.log("valid_kappa_unweighted", self.valid_kappa_unweighted.compute(),
                  on_epoch=True, on_step=False, prog_bar=True)
        self.valid_kappa_lin_weighted.update(predicted_labels, true_labels)
        self.log("valid_kappa_lin_weighted", self.valid_kappa_lin_weighted.compute(),
                  on_epoch=True, on_step=False, prog_bar=True)
        self.valid_kappa_sq_weighted.update(predicted_labels, true_labels)
        self.log("valid_kappa_sq_weighted", self.valid_kappa_sq_weighted.compute(),
                  on_epoch=True, on_step=False, prog_bar=True)
        self.valid_auc.update(predicted_labels, true_labels)
        self.log("valid_auc", self.valid_auc.compute(),
                  on_epoch=True, on_step=False, prog_bar=True)

    def test_step(self, batch, batch_idx):
        _, true_labels, predicted_labels = self._shared_step(batch)
        self.test_mae(predicted_labels, true_labels)
        self.test_specificity(predicted_labels, true_labels)
        self.log("test_mae", self.test_mae, on_epoch=True, on_step=False)

        #self.log("sensitivities", self.test_sensitivity, on_epoch=True , on_step=False)
        #self.log("specificities", self.test_specificity, on_epoch=True , on_step=False)



    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

### Assigning data

Initializing the labels (`pd.DataFrame`) and features (`pd.Series`)

Taking first $n$ observations.

Using PCA result `X_reduced` instead of `X_sc` decreases the accuracy only slightly (~2%), due to the information loss.

In [ ]:


data_labels = response
data_features = pd.DataFrame(X_sc)
data_features_two_class = xres
data_labels_two_class = yres


Determining the baseline MAE. After training the validation MAE should be much lower than this.

In [ ]:
# avg_prediction = np.median(data_labels.values)  # median minimizes MAE
# baseline_mae = np.mean(np.abs(data_labels.values - avg_prediction))
# print(f'Baseline MAE: {baseline_mae:.2f}')

### DataSet
Creating the custom dataset class (Extending the Dataset of Pytorch)

In [ ]:
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self, feature_array, label_array, dtype=np.float32):
        self.features = feature_array.astype(dtype)
        self.labels = label_array

    def __getitem__(self, index):
        inputs = self.features[index]
        label = self.labels[index]
        return inputs, label

    def __len__(self):
        return self.features.shape[0]

### DataModule

Extending the `LightningDataModule` class (implementing the custom `setup()` method along with the respective DataLoaders)

*Uses* the previously defined `Dataset` class.

In [ ]:
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader


class DataModule(pl.LightningDataModule):
    def __init__(self, features, labels, batch_size):
        super().__init__()
        self.data_labels = labels
        self.data_features = features
        self.batch_size = batch_size


    def setup(self, stage=None):
        # data_df = pd.read_csv(
        #     os.path.join(self.data_path, 'cement_strength.csv'))
        # data_df["response"] = data_df["response"]-1  # labels should start at 0
        #self.data_labels = labels # data_df["response"]
        #self.data_features = features

        # Split into
        # 70% train, 10% validation, 20% testing

        X_temp, X_test, y_temp, y_test = train_test_split(
            self.data_features.values,
            self.data_labels.values,
            test_size=0.2,
            random_state=1,
            stratify=self.data_labels.values)

        X_train, X_valid, y_train, y_valid = train_test_split(
            X_temp,
            y_temp,
            test_size=0.1,
            random_state=1,
            stratify=y_temp)

        # Standardize features
        sc = StandardScaler()
        X_train_std = sc.fit_transform(X_train)
        X_valid_std = sc.transform(X_valid)
        X_test_std = sc.transform(X_test)

        self.train = MyDataset(X_train_std, y_train)
        self.valid = MyDataset(X_valid_std, y_valid)
        self.test = MyDataset(X_test_std, y_test)

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size,
                          num_workers=NUM_WORKERS,
                          drop_last=True)

    def val_dataloader(self):
        return DataLoader(self.valid, batch_size=self.batch_size,
                          num_workers=NUM_WORKERS)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size,
                          num_workers=NUM_WORKERS)

Setting the random seed and initializing the DataModule

In [ ]:
torch.manual_seed(1)
data_module = DataModule(data_features, data_labels, BATCH_SIZE)
data_module_two_class = DataModule(data_features_two_class, data_labels_two_class, BATCH_SIZE)

## Four Class Case

### PyTorch model -> Lightning Model

Initializing the Pytorch model of the class `MultilayerPerceptron` and feeding it into the Lightning's implementation of multilayer perceptron.

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger


pytorch_model = MultiLayerPerceptron(
    input_size=data_features.shape[1],
    hidden_units=HIDDEN_UNITS,
    num_classes=np.bincount(data_labels).shape[0])

lightning_model = LightningMLP(
    model=pytorch_model,
    learning_rate=LEARNING_RATE)


callbacks = [ModelCheckpoint(
    save_top_k=1, mode="max", monitor="valid_acc")]  # save top 1 model
#logger = CSVLogger(save_dir="logs/", name="mlp-coral-covid")

### Trainer (and Logger)

Defining the Tensorboard Logger, which will format the logs so that they are representable in Tensorboard.

Setting up the `Trainer`.

Fitting the model and tracking the training time.

The `Trainer` Class logs the results to the logger (Directory defined when initializing the logger)

In [ ]:
import time
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_model", log_graph=True)

trainer = pl.Trainer(
    max_epochs=NUM_EPOCHS,
    callbacks=callbacks,
    progress_bar_refresh_rate=5,  # recommended for notebooks
    accelerator="auto",  # Uses GPUs or TPUs if available
    devices="auto",  # Uses all available GPUs/TPUs if applicable
    logger=logger,
    deterministic=True,
    log_every_n_steps=10)

start_time = time.time()
trainer.fit(model=lightning_model, datamodule=data_module)
runtime = (time.time() - start_time)/60
print(f"Training took {runtime:.2f} min in total.")

Adding layers does not impact validation accuracy significantly.

Noise removed with PCA.

Sigmoid has proven to be better than ReLU.

### Copying previous tensorboard logs from my google drive

In [ ]:
#!cp -r drive/MyDrive/Covid_Bachelor/tb_logs/ /content/

In [ ]:
#!pwd

### Plotting the computation graph


In [ ]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/covid')


In [ ]:
import tensorflow as tf

In [ ]:
next(iter(data_module.train_dataloader()))

In [ ]:
writer.add_graph(lightning_model, next(iter(data_module.train_dataloader()))[0] )

In [ ]:
# %reload_ext tensorboard
# %tensorboard --logdir runs

### Plotting the training history in tensorboard


In [ ]:
%reload_ext tensorboard
# %tensorboard --logdir drive/MyDrive/Covid_Bachelor/tb_logs/
%tensorboard --logdir tb_logs

### Finding the best model

**Path to the best found model of the last run:**

In [ ]:
PATH = trainer.checkpoint_callback.best_model_path

In [ ]:
m = torch.load(PATH)

### Accuracy

Best accuracy found for the last model:

In [ ]:
best_found_acc = list(m['callbacks'].values())[0]['best_model_score'].item()

best_found_acc

In [ ]:
best_model = lightning_model.load_from_checkpoint(PATH,model=pytorch_model,
    learning_rate=LEARNING_RATE)

### Sensitivity and Specificity

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc

sens_metric= MySensitivity(num_classes=4)
kappa_unweighted_metric = torchmetrics.CohenKappa(num_classes=4)
kappa_lin_weighted_metric = torchmetrics.CohenKappa(num_classes=4, weights='linear')
kappa_sq_weighted_metric = torchmetrics.CohenKappa(num_classes=4, weights='quadratic')


all_preds = []
all_targets = []

for id, batch in enumerate(data_module.test_dataloader()):
  _, target, preds = best_model._shared_step(batch)
  sens_metric.update(preds,target)
  kappa_unweighted_metric.update(preds,target)
  kappa_lin_weighted_metric.update(preds,target)
  kappa_sq_weighted_metric.update(preds,target)
  #auc_metric.update(preds, target)
  #my_auc_metric.update(preds,target)
  all_preds.extend(preds.tolist())
  all_targets.extend(target.tolist())


sensitivity = sens_metric.compute()
specificity = sens_metric.compute_specificity()
precision = sens_metric.compute_precision()
npv = sens_metric.compute_npv()

kappa_unweighted = kappa_unweighted_metric.compute().item()
kappa_lin_weighted = kappa_lin_weighted_metric.compute().item()
kappa_sq_weighted = kappa_sq_weighted_metric.compute().item()


preds_dummies = pd.get_dummies(pd.Series(all_preds))
targets_dummies = pd.get_dummies(pd.Series(all_targets))

roc_auc = roc_auc_score(targets_dummies, preds_dummies, multi_class="ovo", average="macro")

**Sensitivities**

In [ ]:

# weighted_roc_auc_ovo = roc_auc_score(
#     all_targets_flat, all_preds_flat, multi_class="ovo", average="weighted"
# )


In [ ]:
sensitivity

In [ ]:
plt.bar( sd_classes_reduced,  height = sensitivity)
plt.title("Sensitivities")

**Specificities**

In [ ]:
specificity

In [ ]:
plt.bar( sd_classes_reduced,  height = specificity)
plt.title("Specificities")

**Precisions**

In [ ]:
precision

In [ ]:
plt.bar( sd_classes_reduced,  height = precision)
plt.title("Precisions")

**NPV**

In [ ]:
npv

In [ ]:
plt.bar( sd_classes_reduced,  height = npv)
plt.title("NPV")

In [ ]:
fig, axs = plt.subplots(2, 2)
axs[0, 0].bar( sd_classes_reduced,  height = sensitivity)
axs[0, 0].set_title('Sensitivities')
axs[0, 1].bar( sd_classes_reduced,  height = specificity)
axs[0, 1].set_title('Specificities')
axs[1, 0].bar( sd_classes_reduced,  height = precision)
axs[1, 0].set_title('Precisions')
axs[1, 1].bar( sd_classes_reduced,  height = npv)
axs[1, 1].set_title('NPVs')

plt.tight_layout()
plt.suptitle('CORAL')

**Kappa values**

In [ ]:
ax=plt.axes()
ax.set_facecolor(color='white')
plt.grid(color='gainsboro')

kappas_names = ['Unweighted', 'Linear', 'Quadratic']
kappas_arr = np.array([kappa_unweighted, kappa_lin_weighted, kappa_sq_weighted])

plt.bar( kappas_names,
        height = kappas_arr, color='darkturquoise')
plt.title("Kappas")
plt.savefig("drive/MyDrive/Covid_Bachelor/graphics/results/nns/kappas_nn_4c.png")

### Plotting the computation graph of the best model found

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/best_model')


In [ ]:
writer.add_graph(best_model, next(iter(data_module.train_dataloader()))[0] )

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir runs/best_model

## Two Class Case

### PyTorch model -> Lightning Model

Initializing the Pytorch model of the class `MultilayerPerceptron` and feeding it into the Lightning's implementation of multilayer perceptron.

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger


pytorch_model_two_class = MultiLayerPerceptron(
    input_size=data_features_two_class.shape[1],
    hidden_units=HIDDEN_UNITS,
    num_classes=np.bincount(data_labels_two_class).shape[0])

lightning_model_two_class = LightningMLP(
    model=pytorch_model_two_class,
    learning_rate=LEARNING_RATE)


callbacks_two_class = [ModelCheckpoint(
    save_top_k=1, mode="max", monitor="valid_acc")]  # save top 1 model
#logger = CSVLogger(save_dir="logs/", name="mlp-coral-covid")

### Trainer (and Logger)

Defining the Tensorboard Logger, which will format the logs so that they are representable in Tensorboard.

Setting up the `Trainer`.

Fitting the model and tracking the training time.

The `Trainer` Class logs the results to the logger (Directory defined when initializing the logger)

In [ ]:
import time
from pytorch_lightning.loggers import TensorBoardLogger

logger_two_class = TensorBoardLogger("tb_logs_two_class", name="my_model_two_class", log_graph=True)

trainer_two_class = pl.Trainer(
    max_epochs=NUM_EPOCHS,
    callbacks=callbacks_two_class,
    progress_bar_refresh_rate=5,  # recommended for notebooks
    accelerator="auto",  # Uses GPUs or TPUs if available
    devices="auto",  # Uses all available GPUs/TPUs if applicable
    logger=logger_two_class,
    deterministic=True,
    log_every_n_steps=10)

start_time = time.time()
trainer_two_class.fit(model=lightning_model_two_class, datamodule=data_module_two_class)
runtime = (time.time() - start_time)/60
print(f"Training took {runtime:.2f} min in total.")

Adding layers does not impact validation accuracy significantly.

Noise removed with PCA.

Sigmoid has proven to be better than ReLU.

### Plotting the computation graph


### Plotting the training history in tensorboard


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir tb_logs_two_class

### Finding the best model

**Path to the best found model of the last run:**

In [ ]:
PATH_two_class = trainer_two_class.checkpoint_callback.best_model_path

In [ ]:
m_two_class= torch.load(PATH_two_class)

### Accuracy

Best accuracy found for the last model:

In [ ]:
best_found_acc_two_class = list(m_two_class['callbacks'].values())[0]['best_model_score'].item()

best_found_acc_two_class

In [ ]:
best_model_two_class = lightning_model_two_class.load_from_checkpoint(PATH_two_class,model=pytorch_model_two_class,
    learning_rate=LEARNING_RATE)

### Sensitivity and Specificity

In [ ]:
np.sum(response_two_class == 0) / len(response_two_class)

In [ ]:
sens_metric_two_class= MySensitivity(num_classes=2)
kappa_unweighted_metric_two_class = torchmetrics.CohenKappa(num_classes=2)
kappa_lin_weighted_metric_two_class = torchmetrics.CohenKappa(num_classes=2, weights='linear')
kappa_sq_weighted_metric_two_class = torchmetrics.CohenKappa(num_classes=2, weights='quadratic')
#auc_metric_two_class = torchmetrics.AUC(reorder=True)


all_preds_two_class = []
all_targets_two_class = []


for id, batch in enumerate(data_module_two_class.test_dataloader()):

  _, target, preds = best_model_two_class._shared_step(batch)

  #print('fst',any(preds == 1))
  #print('snd',target)
  sens_metric_two_class.update(preds,target)

  kappa_unweighted_metric_two_class.update(preds,target)
  kappa_lin_weighted_metric_two_class.update(preds,target)
  kappa_sq_weighted_metric_two_class.update(preds,target)
  all_preds_two_class.extend(preds.tolist())
  all_targets_two_class.extend(target.tolist())


sensitivity_two_class = sens_metric_two_class.compute()
specificity_two_class = sens_metric_two_class.compute_specificity()
precision_two_class = sens_metric_two_class.compute_precision()
npv_two_class = sens_metric_two_class.compute_npv()

kappa_unweighted_two_class = kappa_unweighted_metric_two_class.compute().item()
kappa_lin_weighted_two_class = kappa_lin_weighted_metric_two_class.compute().item()
kappa_sq_weighted_two_class = kappa_sq_weighted_metric_two_class.compute().item()

fpr_unsorted,tpr_unsorted,thresholds_unsorted=roc_curve(all_targets_two_class,all_preds_two_class,pos_label=1)

auc_two_class=auc(fpr_unsorted,tpr_unsorted)



In [ ]:

#plt.plot(fpr_unsorted, tpr_unsorted)

In [ ]:
np.sum(np.array(all_targets_two_class) == np.array(all_preds_two_class)) / len(all_targets_two_class)

**Sensitivities**

In [ ]:
sensitivity_two_class

In [ ]:
plt.bar( sd_classes_reduced_two_class,  height = sensitivity_two_class)
plt.title("Sensitivities")

**Specificities**

In [ ]:
specificity_two_class

In [ ]:
plt.bar( sd_classes_reduced_two_class,  height = specificity_two_class)
plt.title("Specificities")

**Precisions**

In [ ]:
precision_two_class

In [ ]:
plt.bar( sd_classes_reduced_two_class,  height = precision_two_class)
plt.title("Precisions")

**NPV**

In [ ]:
npv_two_class

In [ ]:
plt.bar( sd_classes_reduced_two_class,  height = npv_two_class)
plt.title("NPV")

In [ ]:
fig, axs = plt.subplots(2, 2)
axs[0, 0].bar( sd_classes_reduced_two_class,  height = sensitivity_two_class)
axs[0, 0].set_title('Sensitivities')
axs[0, 1].bar( sd_classes_reduced_two_class,  height = specificity_two_class)
axs[0, 1].set_title('Specificities')
axs[1, 0].bar( sd_classes_reduced_two_class,  height = precision_two_class)
axs[1, 0].set_title('Precisions')
axs[1, 1].bar( sd_classes_reduced_two_class,  height = npv_two_class)
axs[1, 1].set_title('NPVs')

plt.tight_layout()
plt.suptitle('CORAL')

**Kappa values**

In [ ]:
ax=plt.axes()
ax.set_facecolor(color='white')
plt.grid(color='gainsboro')

kappas_names = ['Unweighted', 'Linear', 'Quadratic']
kappas_arr_two_class = np.array([kappa_unweighted_two_class, kappa_lin_weighted_two_class, kappa_sq_weighted_two_class])

plt.bar( kappas_names,
        height = kappas_arr_two_class, color='darkturquoise')
plt.title("Kappas")
plt.savefig("drive/MyDrive/Covid_Bachelor/graphics/results/nns/kappas_nn_2c.png")

**All Kappas equal.**

# Hyperparameter Optimization (Learning Rate and Batch Size)

In [ ]:
# pip install optuna

In [ ]:
#import optuna
#from optuna.integration import PyTorchLightningPruningCallback

In [ ]:
# EPOCHS = 10

# DATA_BASEPATH = "./data"

Implementing custom pytorch module (multilayer perceptron)

Hidden units are given over an iterable and activation function

In [ ]:
# import torch
# from coral_pytorch.layers import CoralLayer


# # Regular PyTorch Module
# class MultiLayerPerceptron(torch.nn.Module):
#     def __init__(self, input_size, hidden_units, num_classes):
#         super().__init__()

#         # num_classes is used by the CORAL loss function
#         self.num_classes = num_classes

#         # Initialize MLP layers
#         all_layers = []
#         for hidden_unit in hidden_units:
#             layer = torch.nn.Linear(input_size, hidden_unit)
#             all_layers.append(layer)
#             all_layers.append(torch.nn.Sigmoid())
#             input_size = hidden_unit

#         # CORAL: output layer -------------------------------------------
#         # Regular classifier would use the following output layer:
#         # output_layer = torch.nn.Linear(hidden_units[-1], num_classes)

#         # We replace it by the CORAL layer:
#         output_layer = CoralLayer(size_in=hidden_units[-1],
#                                   num_classes=num_classes)
#         # ----------------------------------------------------------------

#         all_layers.append(output_layer)
#         self.model = torch.nn.Sequential(*all_layers)

#     def forward(self, x):
#         x = self.model(x)
#         return x

In [ ]:
# from coral_pytorch.losses import coral_loss
# from coral_pytorch.dataset import levels_from_labelbatch
# from coral_pytorch.dataset import proba_to_label

# import pytorch_lightning as pl
# import torchmetrics


# # LightningModule that receives a PyTorch model as input
# class LightningMLP(pl.LightningModule):
#     def __init__(self, model, learning_rate):
#         super().__init__()

#         self.learning_rate = learning_rate
#         # The inherited PyTorch module
#         self.model = model

#         # Save settings and hyperparameters to the log directory
#         # but skip the model parameters
#         self.save_hyperparameters(ignore=['model'])

#         # Set up attributes for computing the MAE
#         self.train_mae = torchmetrics.MeanAbsoluteError()
#         self.train_acc = torchmetrics.Accuracy()
#         self.valid_mae = torchmetrics.MeanAbsoluteError()
#         self.valid_acc = torchmetrics.Accuracy()
#         self.test_mae = torchmetrics.MeanAbsoluteError()


#     # Defining the forward method is only necessary
#     # if you want to use a Trainer's .predict() method (optional)
#     def forward(self, x):
#         return self.model(x)

#     # A common forward step to compute the loss and labels
#     # this is used for training, validation, and testing below
#     def _shared_step(self, batch):
#         features, true_labels = batch

#         # Convert class labels for CORAL ------------------------
#         levels = levels_from_labelbatch(
#             true_labels, num_classes=self.model.num_classes)
#         # -------------------------------------------------------

#         logits = self(features)

#         # CORAL Loss --------------------------------------------
#         # A regular classifier uses:
#         # loss = torch.nn.functional.cross_entropy(logits, true_labels)
#         loss = coral_loss(logits, levels.type_as(logits))
#         # -------------------------------------------------------

#         # CORAL Prediction to label -----------------------------
#         # A regular classifier uses:
#         # predicted_labels = torch.argmax(logits, dim=1)
#         probas = torch.sigmoid(logits)
#         predicted_labels = proba_to_label(probas)
#         # -------------------------------------------------------
#         return loss, true_labels, predicted_labels

#     def training_step(self, batch, batch_idx):
#         loss, true_labels, predicted_labels = self._shared_step(batch)
#         # self.log("train_loss", loss)
#         self.train_mae(predicted_labels, true_labels)
#         # self.log("train_mae", self.train_mae, on_epoch=True, on_step=False)
#         self.train_acc(predicted_labels, true_labels)
#         # self.log("train_acc", self.train_acc, on_epoch=True, on_step=False)
#         return loss  # this is passed to the optimzer for training

#     def validation_step(self, batch, batch_idx):
#         loss, true_labels, predicted_labels = self._shared_step(batch)
#         # self.log("valid_loss", loss)
#         self.valid_mae(predicted_labels, true_labels)
#         # self.log("valid_mae", self.valid_mae,
#         #         on_epoch=True, on_step=False, prog_bar=True)
#         self.valid_acc(predicted_labels, true_labels)
#         self.log("valid_acc", self.valid_acc,
#                  on_epoch=True, on_step=False, prog_bar=True)

#     def test_step(self, batch, batch_idx):
#         _, true_labels, predicted_labels = self._shared_step(batch)
#         self.test_mae(predicted_labels, true_labels)
#         # self.log("test_mae", self.test_mae, on_epoch=True, on_step=False)

#     def configure_optimizers(self):
#         optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
#         return optimizer

In [ ]:
# from pytorch_lightning.loggers.base import LightningLoggerBase

# class DictLogger(LightningLoggerBase):
#     """PyTorch Lightning `dict` logger."""

#     def __init__(self, version):
#         super(DictLogger, self).__init__()
#         self.metrics = []
#         self._version = version

#     def log_metrics(self, metrics, step=None):
#         self.metrics.append(metrics)

#     @property
#     def version(self):
#         return self._version

#     @property
#     def experiment(self):
#         """Return the experiment object associated with this logger."""

#     def log_hyperparams(self, params):
#         """
#         Record hyperparameters.
#         Args:
#             params: :class:`~argparse.Namespace` containing the hyperparameters
#         """

#     @property
#     def name(self):
#         """Return the experiment name."""
#         return 'optuna'

In [ ]:
# import time

# def objective(trial):
#   # PyTorch Lightning will try to restore model parameters from previous trials if checkpoint
#   # filenames match. Therefore, the filenames for each trial must be made unique.
#   checkpoint_callback = pl.callbacks.ModelCheckpoint(
#       os.path.join("trial_{}".format(trial.number)), monitor="accuracy"
#   )

#   # The default logger in PyTorch Lightning writes to event files to be consumed by
#   # TensorBoard. We create a simple logger instead that holds the log in memory so that the
#   # final accuracy can be obtained after optimization. When using the default logger, the
#   # final accuracy could be stored in an attribute of the `Trainer` instead.
#   logger = DictLogger(trial.number)


#   trainer = pl.Trainer(
#       max_epochs=10,
#       checkpoint_callback=checkpoint_callback,
#       progress_bar_refresh_rate=5,  # recommended for notebooks
#       accelerator="auto",  # Uses GPUs or TPUs if available
#       devices="auto",  # Uses all available GPUs/TPUs if applicable
#       logger=logger,
#       deterministic=True,
#       callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_accuracy")],
#       )

#   pytorch_model = MultiLayerPerceptron(
#       input_size=data_features.shape[1],
#       hidden_units=[116, 116, 58],
#       num_classes=np.bincount(data_labels).shape[0])

#   lr = trial.suggest_uniform("learning_rate", 5e-3,5e-2)
#   batch_size = trial.suggest_categorical("batch_size", [256, 512, 1024, 2048])

#   lightning_model = LightningMLP(
#       model=pytorch_model,
#       learning_rate=lr)

#   data_module = DataModule(data_features, data_labels, batch_size)

#   trainer.fit(model=lightning_model, datamodule=data_module )
#   print("METRICS::::::", logger.metrics)
#   return logger.metrics[-1]["valid_acc"]


In [ ]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=10)

# print("Number of finished trials: {}".format(len(study.trials)))

# print("Best trial:")
# trial = study.best_trial

# print("  Value: {}".format(trial.value))

# print("  Params: ")
# for key, value in trial.params.items():
#   print("    {}: {}".format(key, value))

After 10 trials, 10 epochs each, these were the results:

Best trial:

  Value: 0.4505000114440918

  Params:

* learning_rate: 0.02506310936524591

* batch_size: 1024

In [ ]:
# study.trials

# Visualizing Activations and gradients of each layer

In [ ]:
##############################################################
import seaborn as sns
import torch.utils.data as data


def plot_dists(val_dict, color="C0", xlabel=None, stat="count", use_kde=True):
    columns = len(val_dict)
    fig, ax = plt.subplots(1, columns, figsize=(columns * 3, 2.5))
    fig_index = 0
    for key in sorted(val_dict.keys()):
        key_ax = ax[fig_index % columns]
        sns.histplot(
            val_dict[key],
            ax=key_ax,
            color=color,
            bins=50,
            stat=stat,
            kde=use_kde and ((val_dict[key].max() - val_dict[key].min()) > 1e-8),
        )  # Only plot kde if there is variance
        hidden_dim_str = (
            r"(%i $\to$ %i)" % (val_dict[key].shape[1], val_dict[key].shape[0]) if len(val_dict[key].shape) > 1 else ""
        )
        key_ax.set_title(f"{key} {hidden_dim_str}")
        if xlabel is not None:
            key_ax.set_xlabel(xlabel)
        fig_index += 1
    fig.subplots_adjust(wspace=0.4)
    return fig


##############################################################


def visualize_weight_distribution(model, color="C0"):
    weights = {}
    for name, param in model.named_parameters():
        if name.endswith(".bias"):
            continue
        key_name = f"Layer {name.split('.')[1]}"
        weights[key_name] = param.detach().view(-1).cpu().numpy()

    # Plotting
    fig = plot_dists(weights, color=color, xlabel="Weight vals")
    fig.suptitle("Weight distribution", fontsize=14, y=1.05)
    plt.show()
    plt.close()


##############################################################


def visualize_gradients(model, color="C0", print_variance=False):
    """
    Args:
        net: Object of class BaseNetwork
        color: Color in which we want to visualize the histogram (for easier separation of activation functions)
    """
    model.eval()
    small_loader = data.DataLoader(train_set, batch_size=1024, shuffle=False)
    imgs, labels = next(iter(small_loader))
    imgs, labels = imgs.to(device), labels.to(device)

    # Pass one batch through the network, and calculate the gradients for the weights
    model.zero_grad()
    preds = model(imgs)
    loss = F.cross_entropy(preds, labels)  # Same as nn.CrossEntropyLoss, but as a function instead of module
    loss.backward()
    # We limit our visualization to the weight parameters and exclude the bias to reduce the number of plots
    grads = {
        name: params.grad.view(-1).cpu().clone().numpy()
        for name, params in model.named_parameters()
        if "weight" in name
    }
    model.zero_grad()

    # Plotting
    fig = plot_dists(grads, color=color, xlabel="Grad magnitude")
    fig.suptitle("Gradient distribution", fontsize=14, y=1.05)
    plt.show()
    plt.close()

    if print_variance:
        for key in sorted(grads.keys()):
            print(f"{key} - Variance: {np.var(grads[key])}")


##############################################################


def visualize_activations(model, color="C0", print_variance=False):
    model.eval()
    small_loader = data.DataLoader(train_set, batch_size=1024, shuffle=False)
    imgs, labels = next(iter(small_loader))
    imgs, labels = imgs.to(device), labels.to(device)

    # Pass one batch through the network, and calculate the gradients for the weights
    feats = imgs.view(imgs.shape[0], -1)
    activations = {}
    with torch.no_grad():
        for layer_index, layer in enumerate(model.layers):
            feats = layer(feats)
            if isinstance(layer, nn.Linear):
                activations[f"Layer {layer_index}"] = feats.view(-1).detach().cpu().numpy()

    # Plotting
    fig = plot_dists(activations, color=color, stat="density", xlabel="Activation vals")
    fig.suptitle("Activation distribution", fontsize=14, y=1.05)
    plt.show()
    plt.close()

    if print_variance:
        for key in sorted(activations.keys()):
            print(f"{key} - Variance: {np.var(activations[key])}")


##############################################################

In [ ]:
#visualize_weight_distribution(lightning_model)
#visualize_gradients(lightning_model)
#visualize_activations(lightning_model, print_variance=True)

In [ ]:
def visualize_weight_distribution(model, color="C0"):
    weights = {}
    for name, param in model.named_parameters():
        if name.endswith(".bias"):
            continue
        key_name = f"Layer {name.split('.')[1]}"
        weights[key_name] = param.detach().view(-1).cpu().numpy()

    # Plotting
    fig = plot_dists(weights, color=color, xlabel="Weight vals")
    fig.suptitle("Weight distribution", fontsize=14, y=1.05)
    plt.show()
    plt.close()

visualize_weight_distribution(pytorch_model)

In [ ]:
# weights = {}
# for name, param in pytorch_model.named_parameters():
#   if name.endswith(".bias"):
#       continue
#   key_name = f"Layer {name.split('.')[1]}"
#   weights[key_name] = param.detach().view(-1).cpu().numpy()



# Naive Neural Network Classification



### Multilayer Perceptron


In [ ]:
# Regular PyTorch Module
class MultiLayerPerceptronClassic(torch.nn.Module):
    def __init__(self, input_size, hidden_units, num_classes):
        super().__init__()

        # num_classes is used by the CORAL loss function
        self.num_classes = num_classes

        # Initialize MLP layers
        all_layers = []
        for hidden_unit in hidden_units:
            layer = torch.nn.Linear(input_size, hidden_unit)
            all_layers.append(layer)
            all_layers.append(torch.nn.Sigmoid())
            input_size = hidden_unit

        # CORAL: output layer -------------------------------------------
        # Regular classifier would use the following output layer:
        output_layer = torch.nn.Linear(hidden_units[-1], num_classes)

        # We replace it by the CORAL layer:
        #output_layer = CoralLayer(size_in=hidden_units[-1],
        #                          num_classes=num_classes)
        # ----------------------------------------------------------------

        all_layers.append(output_layer)
        self.model = torch.nn.Sequential(*all_layers)

    def forward(self, x):
        x = self.model(x)
        return x

### Lightning Module

In [ ]:
# LightningModule that receives a PyTorch model as input
class LightningMLP_Classic(pl.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()

        self.learning_rate = learning_rate
        # The inherited PyTorch module
        self.model = model

        # Save settings and hyperparameters to the log directory
        # but skip the model parameters
        self.save_hyperparameters(ignore=['model'])

        # Set up attributes for computing the MAE
        self.train_mae = torchmetrics.MeanAbsoluteError()
        self.train_acc = torchmetrics.Accuracy()
        self.valid_mae = torchmetrics.MeanAbsoluteError()
        self.valid_acc = torchmetrics.Accuracy()
        self.test_mae = torchmetrics.MeanAbsoluteError()
        self.valid_sensitivity = MySensitivity(num_classes=self.model.num_classes)
        self.valid_kappa_unweighted = torchmetrics.CohenKappa(num_classes=self.model.num_classes)
        self.valid_kappa_lin_weighted = torchmetrics.CohenKappa(num_classes=self.model.num_classes, weights='linear')
        self.valid_kappa_sq_weighted = torchmetrics.CohenKappa(num_classes=self.model.num_classes, weights='quadratic')


    # Defining the forward method is only necessary
    # if you want to use a Trainer's .predict() method (optional)
    def forward(self, x):
        return self.model(x)

    # A common forward step to compute the loss and labels
    # this is used for training, validation, and testing below
    def _shared_step(self, batch):
        features, true_labels = batch

        # Convert class labels for CORAL ------------------------
        levels = levels_from_labelbatch(
            true_labels, num_classes=self.model.num_classes)
        # -------------------------------------------------------

        logits = self(features)

        # CORAL Loss --------------------------------------------
        # A regular classifier uses:
        loss = torch.nn.functional.cross_entropy(logits, true_labels)
        # loss = coral_loss(logits, levels.type_as(logits))
        # -------------------------------------------------------

        # CORAL Prediction to label -----------------------------
        # A regular classifier uses:
        predicted_labels = torch.argmax(logits, dim=1)
        #probas = torch.sigmoid(logits)
        #predicted_labels = proba_to_label(probas)
        # -------------------------------------------------------
        return loss, true_labels, predicted_labels

    def training_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        self.log("train_loss", loss)
        self.train_mae(predicted_labels, true_labels)
        self.log("train_mae", self.train_mae, on_epoch=True, on_step=False)
        self.train_acc(predicted_labels, true_labels)
        self.log("train_acc", self.train_acc, on_epoch=True, on_step=False)
        return loss  # this is passed to the optimzer for training

    def validation_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        # self.log("valid_loss", loss)
        self.valid_mae(predicted_labels, true_labels)
        #self.log("valid_mae", self.valid_mae,
        #         on_epoch=True, on_step=False, prog_bar=True)
        self.valid_acc(predicted_labels, true_labels)
        self.log("valid_acc", self.valid_acc,
                 on_epoch=True, on_step=False, prog_bar=True)
        self.valid_sensitivity.update(predicted_labels, true_labels)
        self.log("valid_sens_last_class", 1-self.valid_sensitivity.compute()[(self.model.num_classes-1)],
                  on_epoch=True, on_step=False, prog_bar=True)

        # Kappas
        self.valid_kappa_unweighted.update(predicted_labels, true_labels)
        self.log("valid_kappa_unweighted", self.valid_kappa_unweighted.compute(),
                  on_epoch=True, on_step=False, prog_bar=True)
        self.valid_kappa_lin_weighted.update(predicted_labels, true_labels)
        self.log("valid_kappa_lin_weighted", self.valid_kappa_lin_weighted.compute(),
                  on_epoch=True, on_step=False, prog_bar=True)
        self.valid_kappa_sq_weighted.update(predicted_labels, true_labels)
        self.log("valid_kappa_sq_weighted", self.valid_kappa_sq_weighted.compute(),
                  on_epoch=True, on_step=False, prog_bar=True)

    def test_step(self, batch, batch_idx):
        _, true_labels, predicted_labels = self._shared_step(batch)
        self.test_mae(predicted_labels, true_labels)
        self.test_specificity(predicted_labels, true_labels)
        self.log("test_mae", self.test_mae, on_epoch=True, on_step=False)

        #self.log("sensitivities", self.test_sensitivity, on_epoch=True , on_step=False)
        #self.log("specificities", self.test_specificity, on_epoch=True , on_step=False)



    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

### Data module

In [ ]:
torch.manual_seed(1)
data_module_classic = DataModule(data_features, data_labels, BATCH_SIZE)
data_module_classic_two_class = DataModule(data_features_two_class, data_labels_two_class, BATCH_SIZE)

## Four Class Case

### PyTorch model -> Lightning Model

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger


pytorch_model_classic = MultiLayerPerceptronClassic(
    input_size=data_features.shape[1],
    hidden_units=HIDDEN_UNITS,
    num_classes=np.bincount(data_labels).shape[0])

lightning_model_classic = LightningMLP_Classic(
    model=pytorch_model_classic,
    learning_rate=LEARNING_RATE)


callbacks = [ModelCheckpoint(
    save_top_k=1, mode="max", monitor="valid_acc")]  # save top 1 model
#logger = CSVLogger(save_dir="logs/", name="mlp-coral-covid")

### Trainer (and Logger)

Defining the Tensorboard Logger, which will format the logs so that they are representable in Tensorboard.

Setting up the `Trainer`.

Fitting the model and tracking the training time.

The `Trainer` Class logs the results to the logger (Directory defined when initializing the logger)

In [ ]:
import time
from pytorch_lightning.loggers import TensorBoardLogger

logger_classic = TensorBoardLogger("tb_logs_classic", name="my_model_classic", log_graph=True)

trainer_classic = pl.Trainer(
    max_epochs=NUM_EPOCHS,
    callbacks=callbacks,
    progress_bar_refresh_rate=5,  # recommended for notebooks
    accelerator="auto",  # Uses GPUs or TPUs if available
    devices="auto",  # Uses all available GPUs/TPUs if applicable
    logger=logger_classic,
    deterministic=True,
    log_every_n_steps=10)

start_time = time.time()
trainer_classic.fit(model=lightning_model_classic, datamodule=data_module_classic)
runtime = (time.time() - start_time)/60
print(f"Training took {runtime:.2f} min in total.")

### Tensorboard training history

In [ ]:
%reload_ext tensorboard
# %tensorboard --logdir drive/MyDrive/Covid_Bachelor/tb_logs/
%tensorboard --logdir tb_logs_classic

### Finding the best model

**Path to the best found model of the last run:**

In [ ]:
PATH_classic = trainer_classic.checkpoint_callback.best_model_path

In [ ]:
m_classic = torch.load(PATH_classic)

### Accuracy

Best accuracy found for the last model:

In [ ]:
best_found_acc_classic = list(m_classic['callbacks'].values())[0]['best_model_score'].item()

best_found_acc_classic

In [ ]:
best_model_classic = lightning_model_classic.load_from_checkpoint(PATH_classic,model=pytorch_model_classic,
    learning_rate=LEARNING_RATE)

### Sensitivity and Specificity

In [ ]:
sens_metric_classic= MySensitivity(num_classes=4)
kappa_unweighted_metric_classic = torchmetrics.CohenKappa(num_classes=4)
kappa_lin_weighted_metric_classic = torchmetrics.CohenKappa(num_classes=4, weights='linear')
kappa_sq_weighted_metric_classic = torchmetrics.CohenKappa(num_classes=4, weights='quadratic')

all_preds_classic = []
all_targets_classic = [] # in the end same as all_targets

for id, batch in enumerate(data_module_classic.test_dataloader()):
  _, target, preds = best_model_classic._shared_step(batch)
  sens_metric_classic.update(preds,target)
  kappa_unweighted_metric_classic.update(preds,target)
  kappa_lin_weighted_metric_classic.update(preds,target)
  kappa_sq_weighted_metric_classic.update(preds,target)
  #aucmetric_classic.update(preds, target)
  all_preds_classic.extend(preds.tolist())
  all_targets_classic.extend(target.tolist())

sensitivity_classic = sens_metric_classic.compute()
specificity_classic = sens_metric_classic.compute_specificity()
precision_classic = sens_metric_classic.compute_precision()
npv_classic = sens_metric_classic.compute_npv()

kappa_unweighted_classic = kappa_unweighted_metric_classic.compute().item()
kappa_lin_weighted_classic = kappa_lin_weighted_metric_classic.compute().item()
kappa_sq_weighted_classic = kappa_sq_weighted_metric_classic.compute().item()

preds_dummies_classic = pd.get_dummies(pd.Series(all_preds_classic))
targets_dummies_classic = pd.get_dummies(pd.Series(all_targets_classic))

roc_auc_classic = roc_auc_score(targets_dummies_classic, preds_dummies_classic, multi_class="ovo", average="macro")


**Sensitivities**

In [ ]:
sensitivity_classic

In [ ]:
plt.bar( sd_classes_reduced,  height = sensitivity_classic)
plt.title("Sensitivities")

**Specificities**

In [ ]:
specificity_classic

In [ ]:
plt.bar( sd_classes_reduced,  height = specificity_classic)
plt.title("Specificities")

In [ ]:
fig, axs = plt.subplots(2, 2)
axs[0, 0].bar( sd_classes_reduced,  height = sensitivity_classic)
axs[0, 0].set_title('Sensitivities')
axs[0, 1].bar( sd_classes_reduced,  height = specificity_classic)
axs[0, 1].set_title('Specificities')
axs[1, 0].bar( sd_classes_reduced,  height = precision_classic)
axs[1, 0].set_title('Precisions')
axs[1, 1].bar( sd_classes_reduced,  height = npv_classic)
axs[1, 1].set_title('NPVs')

plt.tight_layout()
plt.suptitle('Classic NNs')

In [ ]:
precision

**Kappa values**

In [ ]:
ax=plt.axes()
ax.set_facecolor(color='white')
plt.grid(color='gainsboro')

kappas_names = ['Unweighted', 'Linear', 'Quadratic']
kappas_arr_classic = np.array([kappa_unweighted_classic, kappa_lin_weighted_classic, kappa_sq_weighted_classic])

plt.bar( kappas_names,
        height = kappas_arr_classic, color='darkturquoise')
plt.title("Kappas - Naive")
plt.savefig("drive/MyDrive/Covid_Bachelor/graphics/results/nns/kappas_nn_4c_naive.png")

### Plotting the computation graph of the best model found

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer_classic = SummaryWriter('runs/best_model_classic')


In [ ]:
writer_classic.add_graph(best_model_classic, next(iter(data_module_classic.train_dataloader()))[0] )

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir runs/best_model_classic

## Two Class Case

### PyTorch model -> Lightning Model

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger


pytorch_model_classic_two_class = MultiLayerPerceptronClassic(
    input_size=data_features.shape[1],
    hidden_units=HIDDEN_UNITS,
    num_classes=np.bincount(data_labels_two_class).shape[0])

lightning_model_classic_two_class = LightningMLP_Classic(
    model=pytorch_model_classic_two_class,
    learning_rate=LEARNING_RATE)


callbacks_two_class = [ModelCheckpoint(
    save_top_k=1, mode="max", monitor="valid_acc")]  # save top 1 model
#logger = CSVLogger(save_dir="logs/", name="mlp-coral-covid")

### Trainer (and Logger)

Defining the Tensorboard Logger, which will format the logs so that they are representable in Tensorboard.

Setting up the `Trainer`.

Fitting the model and tracking the training time.

The `Trainer` Class logs the results to the logger (Directory defined when initializing the logger)

In [ ]:
import time
from pytorch_lightning.loggers import TensorBoardLogger

logger_classic_two_class = TensorBoardLogger("tb_logs_classic_two_class", name="my_model_classic_two_class", log_graph=True)

trainer_classic_two_class = pl.Trainer(
    max_epochs=NUM_EPOCHS,
    callbacks=callbacks_two_class,
    progress_bar_refresh_rate=5,  # recommended for notebooks
    accelerator="auto",  # Uses GPUs or TPUs if available
    devices="auto",  # Uses all available GPUs/TPUs if applicable
    logger=logger_classic_two_class,
    deterministic=True,
    log_every_n_steps=10)

start_time = time.time()
trainer_classic_two_class.fit(model=lightning_model_classic_two_class, datamodule=data_module_classic_two_class)
runtime = (time.time() - start_time)/60
print(f"Training took {runtime:.2f} min in total.")

### Tensorboard training history

In [ ]:
%reload_ext tensorboard
# %tensorboard --logdir drive/MyDrive/Covid_Bachelor/tb_logs/
%tensorboard --logdir tb_logs_classic_two_class

### Finding the best model

**Path to the best found model of the last run:**

In [ ]:
PATH_classic_two_class = trainer_classic_two_class.checkpoint_callback.best_model_path

In [ ]:
m_classic_two_class = torch.load(PATH_classic_two_class)

### Accuracy

Best accuracy found for the last model:

In [ ]:
best_found_acc_classic_two_class = list(m_classic_two_class['callbacks'].values())[0]['best_model_score'].item()

best_found_acc_classic_two_class

In [ ]:
best_model_classic_two_class = lightning_model_classic_two_class.load_from_checkpoint(PATH_classic_two_class,
                                                                                      model=pytorch_model_classic_two_class,
                                                                                      learning_rate=LEARNING_RATE)

### Sensitivity and Specificity

In [ ]:
sens_metric_classic_two_class = MySensitivity(num_classes=2)
kappa_unweighted_metric_classic_two_class = torchmetrics.CohenKappa(num_classes=2)
kappa_lin_weighted_metric_classic_two_class = torchmetrics.CohenKappa(num_classes=2, weights='linear')
kappa_sq_weighted_metric_classic_two_class = torchmetrics.CohenKappa(num_classes=2, weights='quadratic')
acc_metric_classic_two_class = torchmetrics.Accuracy(num_classes=2)

all_preds_classic_two_class = []
all_targets_classic_two_class = []

for id, batch in enumerate(data_module_classic_two_class.test_dataloader()):
  _, target, preds = best_model_classic_two_class._shared_step(batch)
  sens_metric_classic_two_class.update(preds,target)
  kappa_unweighted_metric_classic_two_class.update(preds,target)
  kappa_lin_weighted_metric_classic_two_class.update(preds,target)
  kappa_sq_weighted_metric_classic_two_class.update(preds,target)
  all_preds_classic_two_class.extend(preds.tolist())
  all_targets_classic_two_class.extend(target.tolist())
  acc_metric_classic_two_class.update(preds,target)

sensitivity_classic_two_class = sens_metric_classic_two_class.compute()
specificity_classic_two_class = sens_metric_classic_two_class.compute_specificity()
precision_classic_two_class = sens_metric_classic_two_class.compute_precision()
npv_classic_two_class = sens_metric_classic_two_class.compute_npv()

kappa_unweighted_classic_two_class = kappa_unweighted_metric_classic_two_class.compute().item()
kappa_lin_weighted_classic_two_class = kappa_lin_weighted_metric_classic_two_class.compute().item()
kappa_sq_weighted_classic_two_class = kappa_sq_weighted_metric_classic_two_class.compute().item()
acc_classic_two_class = acc_metric_classic_two_class.compute().item()

fpr_unsorted_classic,tpr_unsorted_classic, _ =roc_curve(all_targets_classic_two_class,all_preds_classic_two_class,pos_label=1)

auc_classic_two_class=auc(fpr_unsorted_classic,tpr_unsorted_classic)

In [ ]:
acc_classic_two_class

In [ ]:
best_found_acc_classic_two_class

**Sensitivities**

In [ ]:
sensitivity_classic_two_class

In [ ]:
plt.bar( sd_classes_reduced_two_class,  height = sensitivity_classic_two_class)
plt.title("Sensitivities")

**Specificities**

In [ ]:
specificity_classic_two_class

In [ ]:
plt.bar( sd_classes_reduced_two_class,  height = specificity_classic_two_class)
plt.title("Specificities")

In [ ]:
fig, axs = plt.subplots(2, 2)
axs[0, 0].bar( sd_classes_reduced_two_class,  height = sensitivity_classic_two_class)
axs[0, 0].set_title('Sensitivities')
axs[0, 1].bar( sd_classes_reduced_two_class,  height = specificity_classic_two_class)
axs[0, 1].set_title('Specificities')
axs[1, 0].bar( sd_classes_reduced_two_class,  height = precision_classic_two_class)
axs[1, 0].set_title('Precisions')
axs[1, 1].bar( sd_classes_reduced_two_class,  height = npv_classic_two_class)
axs[1, 1].set_title('NPVs')

plt.tight_layout()
plt.suptitle('Classic NNs')

In [ ]:
precision_two_class

**Kappa values**

In [ ]:
ax=plt.axes()
ax.set_facecolor(color='white')
plt.grid(color='gainsboro')

kappas_names = ['Unweighted', 'Linear', 'Quadratic']
kappas_arr_classic_two_class = np.array([kappa_unweighted_classic_two_class,
                               kappa_lin_weighted_classic_two_class,
                               kappa_sq_weighted_classic_two_class])

plt.bar( kappas_names,
        height = kappas_arr_classic_two_class, color='darkturquoise')
plt.title("Kappas - Naive")
plt.savefig("drive/MyDrive/Covid_Bachelor/graphics/results/nns/kappas_nn_2c_naive.png")

# Formatting the results

## Creating Data Frames

In [ ]:
num_layers_4c_coral = len(HIDDEN_UNITS)
num_layers_4c_classic = len(HIDDEN_UNITS_CLASSIC)

result_df = pd.DataFrame( list(zip(['CORAL NN','Classic NN', 'CORAL NN 2C', 'Classic NN 2C'],
                           [best_found_acc, best_found_acc_classic, best_found_acc_two_class, best_found_acc_classic_two_class],
                           [kappa_unweighted, kappa_unweighted_classic,kappa_unweighted_two_class, kappa_unweighted_classic_two_class],
                           [kappa_lin_weighted, kappa_lin_weighted_classic,kappa_lin_weighted_two_class, kappa_lin_weighted_classic_two_class],
                           [kappa_sq_weighted, kappa_sq_weighted_classic,kappa_sq_weighted_two_class, kappa_sq_weighted_classic_two_class],
                           [sensitivity[3], sensitivity_classic[3],sensitivity_two_class[1], sensitivity_classic_two_class[1]],
                           [precision[3], precision_classic[3],precision_two_class[1], precision_classic_two_class[1]],
                            [roc_auc, roc_auc_classic, auc_two_class, auc_classic_two_class])),
                         columns = ['model',
                                    'accuracy', 'unweighted_kappa', 'lin_kappa',
                                    'quadratic_kappa','sensitivity', 'precision', 'AUC'])

In [ ]:
result_df

## Creating Tables

In [ ]:
result_df.to_latex(float_format="%.2f")

Saving to CSV

In [ ]:
result_df.to_csv('drive/MyDrive/Covid_Bachelor/graphics/results/nns/nn_results_to_check.csv')



to_check = pd.read_csv('drive/MyDrive/Covid_Bachelor/graphics/results/nns/nn_results.csv')

In [ ]:
to_check = to_check.drop(columns=['Unnamed: 0','num_hidden_layers', 'layer_architecture'])

In [ ]:
np.round(np.sum(np.sum(result_df.drop(columns=['model']) - to_check.drop(columns= ['model']))))


## Creating Plots

**Accuracy, Sensitivity, Precision**

In [ ]:
save_figs = True

In [ ]:
plt.figure(figsize=(30*0.393701,10*0.393701))
ax=plt.axes()
ax.set_facecolor(color='white')
plt.grid(color='gainsboro',zorder=0)

metrics_names = ['Accuracy', 'Sensitivity of 51+', 'Precision of 51+' ,
                 'Unweighted Kappa', 'Linear Kappa', 'Quadratic Kappa', 'AUC']
metrics_arr =  np.array([best_found_acc , sensitivity[3], precision[3],
                         kappa_unweighted, kappa_lin_weighted, kappa_sq_weighted, roc_auc])
metrics_arr_classic = np.array([best_found_acc_classic , sensitivity_classic[3],
                                precision_classic[3], kappa_unweighted_classic,
                                kappa_lin_weighted_classic, kappa_sq_weighted_classic, roc_auc_classic])

metrics_arr_two_class =  np.array([best_found_acc_two_class , sensitivity_two_class[1], precision_two_class[1],
                         kappa_unweighted_two_class, kappa_lin_weighted_two_class, kappa_sq_weighted_two_class, auc_two_class])
metrics_arr_classic_two_class = np.array([best_found_acc_classic_two_class , sensitivity_classic_two_class[1],
                                precision_classic_two_class[1], kappa_unweighted_classic_two_class,
                                kappa_lin_weighted_classic_two_class, kappa_sq_weighted_classic_two_class, auc_classic_two_class])

x = np.arange(len(metrics_names))  # the label locations
bar_width = 0.2  # the width of the bars

plt.bar( x-0.3,
        height = metrics_arr_classic, color='mediumseagreen', width = bar_width)
plt.bar( x-0.1,
        height = metrics_arr_classic_two_class, color='forestgreen', width = bar_width)

plt.bar( x+0.1,
        height = metrics_arr, color='dodgerblue', width = bar_width)
plt.bar( x+0.3,
        height = metrics_arr_two_class, color='slateblue', width = bar_width)

plt.xticks(x, metrics_names, rotation=45)

plt.legend([ 'Naive NN', 'Naive NN 2c', 'CORAL NN', 'CORAL NN 2c'], loc='lower left')

plt.xlabel('Metrics')

#plt.title("Metrics")


if save_figs:
    plt.savefig("drive/MyDrive/Covid_Bachelor/graphics/results/nns/metrics_comparison.png",
                bbox_inches='tight', dpi=100)


**Kappa values**

In [ ]:
ax=plt.axes()
ax.set_facecolor(color='white')
plt.grid(color='gainsboro')

kappas_names = ['Unweighted', 'Linear', 'Quadratic']
kappas_arr_classic = np.array([kappa_unweighted_classic, kappa_lin_weighted_classic, kappa_sq_weighted_classic])

x = np.arange(len(kappas_names))  # the label locations
bar_width = 0.2  # the width of the bars

plt.bar( x-0.3,
        height = kappas_arr, color='cyan', width = bar_width)
plt.bar( x-0.1,
        height = kappas_arr_classic, color='darkturquoise', width = bar_width)

plt.bar( x+0.1,
        height = kappas_arr_two_class, color='lightseagreen', width = bar_width)
plt.bar( x+0.3,
        height = kappas_arr_classic_two_class, color='skyblue', width = bar_width)


plt.xticks(x, kappas_names)

plt.legend(['CORAL NN', 'Naive NN', 'CORAL NN 2c', 'Naive NN 2c'], loc='lower right')

plt.title("Kappas")

if save_figs:
    plt.savefig("drive/MyDrive/Covid_Bachelor/graphics/results/nns/kappas_comparison.png")
